In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
from senmodel.model.utils import convert_dense_to_sparse_network, get_model_last_layer
from senmodel.metrics.nonlinearity_metrics import GradientMeanEdgeMetric, PerturbationSensitivityEdgeMetric
from senmodel.metrics.edge_finder import EdgeFinder


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

X = data.drop('income', axis=1)
y = data['income']

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

y = LabelEncoder().fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


In [5]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [7]:
class EnhancedFCN(nn.Module):
    def __init__(self, input_size=14, hidden_sizes=None, output_size=2, dropout_rate=0.3):
        super(EnhancedFCN, self).__init__()
        if hidden_sizes is None:
            hidden_sizes = [128, 64, 32]
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(dropout_rate)

        self.output = nn.Linear(hidden_sizes[2], output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        # x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        # x = self.dropout2(x)

        x = self.fc3(x)
        x = self.relu3(x)
        # x = self.dropout3(x)

        x = self.output(x)
        return x


In [24]:
from copy import deepcopy
dense_model = EnhancedFCN(input_size=X.shape[1])

sparse_model = convert_dense_to_sparse_network(deepcopy( dense_model))

In [25]:
ze = torch.zeros((1, X.shape[1]))
dense_model(ze), sparse_model(ze)

(tensor([[-0.1328, -0.0299]], grad_fn=<AddmmBackward0>),
 tensor([[-0.1328, -0.0299]], grad_fn=<AsStridedBackward0>))

In [26]:
aaa = []
for batch, label in val_loader:
    # print(sparse_model(batch))
    aaa.append(sparse_model(batch))

In [27]:
layer = get_model_last_layer(sparse_model)
layer.replace_many([0],[0])

In [28]:
aaa, sparse_model(ze)

([tensor([[-0.1362, -0.0227],
          [-0.1286, -0.0784],
          [-0.1356, -0.0376],
          ...,
          [-0.2246,  0.0251],
          [-0.1107, -0.0863],
          [-0.1766, -0.0806]], grad_fn=<AsStridedBackward0>),
  tensor([[-0.1107, -0.0206],
          [-0.1299, -0.0209],
          [-0.3321, -0.0567],
          ...,
          [-0.1511, -0.0109],
          [-0.1070, -0.0416],
          [-0.1819, -0.0564]], grad_fn=<AsStridedBackward0>),
  tensor([[-0.1732, -0.0333],
          [-0.1442, -0.0279],
          [-0.0953, -0.0660],
          ...,
          [-0.1258, -0.0393],
          [-0.1331, -0.0158],
          [-0.1694, -0.0059]], grad_fn=<AsStridedBackward0>),
  tensor([[-0.2321, -0.0086],
          [-0.1885, -0.0315],
          [-0.1839, -0.0473],
          ...,
          [-0.1706, -0.0226],
          [-0.1609, -0.0700],
          [-0.1546, -0.0237]], grad_fn=<AsStridedBackward0>),
  tensor([[-0.1881, -0.0141],
          [-0.1390, -0.0453],
          [-0.1627, -0.0222],
  

In [29]:
print(aaa[0])

tensor([[-0.1362, -0.0227],
        [-0.1286, -0.0784],
        [-0.1356, -0.0376],
        ...,
        [-0.2246,  0.0251],
        [-0.1107, -0.0863],
        [-0.1766, -0.0806]], grad_fn=<AsStridedBackward0>)


In [30]:

for i, (batch, label) in enumerate(val_loader):
    # print(sparse_model(batch))
    print(sparse_model(batch) - aaa[i])

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<SubBackward0>)
tensor([[ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00],
        ...,
        [ 0.0000e+00,  0.0000e+00],
        [-7.4506e-09,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00]], grad_fn=<SubBackward0>)
tensor([[0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [7.4506e-09, 0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00]], grad_fn=<SubBackward0>)
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<SubBackward0>)
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]], grad_fn=<SubBackward0>)
tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,